In [4]:
# ------------------------------
# Step 0: Imports & Environment Check
# ------------------------------
import os
import sys
import pandas as pd
import numpy as np
import mlflow

print("Python version:", sys.version)
print("MLflow version:", mlflow.__version__)
print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)


Python version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
MLflow version: 3.3.1
Pandas version: 2.2.2
Numpy version: 1.26.4
